In [182]:
import json
from copy import copy
import openai
import traceback

openai.api_key = 

In [179]:
def openai_request(products_to_request):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"""Devolveme solamente el tipo, marca, sabor o ingrediente, cantidad y unidad con el siguiente formato '{{TIPO:MARCA:SABOR_O_INGREDIENTE:CANTIDAD:UNIDAD}}'. Si no encontras la marca, escribí 'None'. En el mismo orden, para el siguiente texto:\n{products_to_request}""",
        temperature=0.6,
        max_tokens=500,
        top_p=0.6,
        frequency_penalty=0,
        presence_penalty=0)
    result = response.choices[0].text
    result = list(filter(lambda each: each != '' and each[0] == '{', result.split('\n')))[0].split('},')
    return list(map(lambda each: each.strip(' {}').split(':'), result))

In [188]:
openai_data

['ENDULZANTE', 'XILITOL', '100', 'GR']

In [186]:
json_file = open(f"results/dietetica_rojas/almacen.json", "r")
raw_products = json.load(json_file)

#tries = 2
step = 15
products = []
try:
    for i in range(0, len(raw_products), step):
        # if tries == 0:
        #     break
        current_raw_products = raw_products[i:i+step]
        raw_products_names_to_request = '\n'.join(map(lambda raw_product: raw_product['item'], current_raw_products))
        for raw_product, openai_data in zip(current_raw_products, openai_request(raw_products_names_to_request)):
            product = copy(raw_product)
            product['type'] = openai_data[0].lower()
            product['brand'] = openai_data[1].lower()
            product['subtype'] = openai_data[2].lower()
            product['measurement'] = ''.join(filter(str.isdigit, openai_data[3]))
            product['unit'] = openai_data[4].lower()
            products.append(product)
        # tries = tries - 1
except Exception as error:
    print(openai_data)
    traceback.print_tb(error.__traceback__)

json_file.close()
json_file2 = open(f"results/dietetica_rojas/almacen2.json", "w")
json_file2.write(json.dumps(products))
json_file2.close()

['ENDULZANTE', 'XILITOL', '100', 'GR']


  File "/tmp/ipykernel_74105/2946560680.py", line 19, in <module>
    product['unit'] = openai_data[4].lower()
